In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Load pictures and labels:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import to_categorical

def load_pictures(link):
    with open(link, 'rb') as f:
        f.read(16)
        buffer = f.read()
        pictures = np.frombuffer(buffer, dtype=np.uint8).reshape(-1, 28, 28)
    return pictures

def load_labels(link):
    with open(link, 'rb') as f:
        f.read(8)
        labels = np.frombuffer(f.read(), dtype=np.uint8)
    return labels

train_images = load_pictures('/content/drive/MyDrive/MÁSTER/IC/P1/Train/train-images.idx3-ubyte')
train_labels = load_labels('/content/drive/MyDrive/MÁSTER/IC/P1/Train/train-labels.idx1-ubyte')
evaluation_images = load_pictures('/content/drive/MyDrive/MÁSTER/IC/P1/Evaluation/t10k-images.idx3-ubyte')
evaluation_labels = load_labels('/content/drive/MyDrive/MÁSTER/IC/P1/Evaluation/t10k-labels.idx1-ubyte')

train_images = train_images / 255.0
evaluation_images = evaluation_images / 255.0
train_images = train_images.reshape(-1, 28 * 28)
evaluation_images = evaluation_images.reshape(-1, 28 * 28)

train_labels = to_categorical(train_labels, num_classes=10)
evaluation_labels = to_categorical(evaluation_labels, num_classes=10)

In [ ]:
#Autoencoder:
input_img = Input(shape=(28 * 28,))
encoded = Dense(256, activation='relu')(input_img)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(28 * 28, activation='sigmoid')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [ ]:
# Training autoencoder:
autoencoder.fit(train_images, train_images, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

Epoch 1/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 0.0922 - val_loss: 0.0347
Epoch 2/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - loss: 0.0301 - val_loss: 0.0220
Epoch 3/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 0.0205 - val_loss: 0.0176
Epoch 4/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0167 - val_loss: 0.0151
Epoch 5/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 0.0145 - val_loss: 0.0134
Epoch 6/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0130 - val_loss: 0.0123
Epoch 7/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0119 - val_loss: 0.0115
Epoch 8/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.0110 - val_loss: 0.0106
Epoch 9/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - loss: 0.0102 - val_loss: 0.0100
Epoch 10/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - loss: 0.0097 - val_loss: 0.0094
Epoch 11/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - loss: 0.0092 - val_loss: 0.0092
Epoch 12/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Neural network:
encoder = Model(input_img, encoded)
train_features = encoder.predict(train_images)
evaluation_features = encoder.predict(evaluation_images)

model = Sequential([
    Dense(128, activation='relu', input_shape=(64,)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Training neural network:
model.fit(train_features, train_labels, epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4819 - loss: 2.1158 - val_accuracy: 0.9023 - val_loss: 0.3336
Epoch 2/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8496 - loss: 0.4779 - val_accuracy: 0.9283 - val_loss: 0.2366
Epoch 3/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8891 - loss: 0.3636 - val_accuracy: 0.9390 - val_loss: 0.1994
Epoch 4/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9069 - loss: 0.3004 - val_accuracy: 0.9482 - val_loss: 0.1708
Epoch 5/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9160 - loss: 0.2746 - val_accuracy: 0.9474 - val_loss: 0.1722
Epoch 6/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9255 - loss: 0.2425 - val_accuracy: 0.9583 - val_loss: 0.1400
Epoch 7/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9295 - loss: 0.2337 - val_accuracy: 0.9549 - val_loss: 0.1482
Epoch 8/50
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9303 - loss: 0.2273 - val_accuracy: 0.

In [ ]:
# Train set :
_, train_accuracy = model.evaluate(train_features, train_labels, verbose=0)
print(f"Train Accuracy: {train_accuracy * 100:.2f}%")

# Evaluation set:
_, evaluation_accuracy = model.evaluate(evaluation_features, evaluation_labels, verbose=0)
print(f"Evaluation Accuracy: {evaluation_accuracy * 100:.2f}%")

Train Accuracy: 97.87%
Evaluation Accuracy: 97.39%
